In [1]:
import torch

import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import cv2
from tqdm import tqdm

In [2]:
raw_data_dir_aptos_train = '/home/chong.tian/hc701/preprocessed/aptos/train'
new_data_dir_aptos_train = '/home/chong.tian/hc701/Pre_train_format/aptos/train'

In [3]:
for i in os.listdir(raw_data_dir_aptos_train):
    data = np.load(os.path.join(raw_data_dir_aptos_train, i), allow_pickle=True).item()
    image = data['image']
    label = data['label']+1
    save_path = os.path.join(new_data_dir_aptos_train, 'class'+str(label))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    cv2.imwrite(os.path.join(save_path, i[:-4]+'.png'), image)

In [4]:
raw_data_dir_aptos_val = '/home/chong.tian/hc701/preprocessed/aptos/val'
new_data_dir_aptos_val = '/home/chong.tian/hc701/Pre_train_format/aptos/val'

In [5]:
for i in os.listdir(raw_data_dir_aptos_val):
    data = np.load(os.path.join(raw_data_dir_aptos_val, i), allow_pickle=True).item()
    image = data['image']
    label = data['label']+1
    save_path = os.path.join(new_data_dir_aptos_val, 'class'+str(label))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    cv2.imwrite(os.path.join(save_path, i[:-4]+'.png'), image)

In [6]:
raw_data_dir_eyepacs_train = '/home/chong.tian/hc701/preprocessed/eyepacs/train'
new_data_dir_eyepacs_train = '/home/chong.tian/hc701/Pre_train_format/eyepacs/train'

In [7]:
for i in os.listdir(raw_data_dir_eyepacs_train):
    data = np.load(os.path.join(raw_data_dir_eyepacs_train, i), allow_pickle=True).item()
    image = data['image']
    label = data['label']+1
    save_path = os.path.join(new_data_dir_eyepacs_train, 'class'+str(label))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # transform image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(save_path, i[:-4]+'.png'), image)

In [8]:
raw_data_dir_eyepacs_val = '/home/chong.tian/hc701/preprocessed/eyepacs/val'
new_data_dir_eyepacs_val = '/home/chong.tian/hc701/Pre_train_format/eyepacs/val'

In [9]:
for i in os.listdir(raw_data_dir_eyepacs_val):
    data = np.load(os.path.join(raw_data_dir_eyepacs_val, i), allow_pickle=True).item()
    image = data['image']
    label = data['label']+1
    save_path = os.path.join(new_data_dir_eyepacs_val, 'class'+str(label))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    # transform image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(save_path, i[:-4]+'.png'), image)

In [1]:
import sys
sys.path.append('/home/chong.tian/hc701/HC701-PROJECT')

import os
import numpy as np

import torch
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset

from hc701fed.dataset.messidor import MESSIDOR

In [2]:
PATH_DATA = '/home/chong.tian/hc701'

In [3]:
MESSIDOR_data_dir_options = {
    'messidor2': os.path.join(PATH_DATA, 'preprocessed/messidor2'),
    'messidor_pairs' : os.path.join(PATH_DATA, 'preprocessed/messidor/messidor_pairs'),
    'messidor_Etienne' : os.path.join(PATH_DATA, 'preprocessed/messidor/messidor_Etienne'),
    'messidor_Brest-without_dilation' : os.path.join(PATH_DATA, 'preprocessed/messidor/messidor_Brest-without_dilation')
}

In [4]:
MESSIDOR_2_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor2'], mode='train', transform_=None)
MESSIDOR_pairs_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_pairs'], mode='train', transform_=None)
MESSIDOR_Etienne_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Etienne'], mode='train', transform_=None)
MESSIDOR_Brest_train = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Brest-without_dilation'], mode='train', transform_=None)

MESSIDOR_2_Val = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor2'], mode='val', transform_=None)
MESSIDOR_pairs_Val = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_pairs'], mode='val', transform_=None)
MESSIDOR_Etienne_Val = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Etienne'], mode='val', transform_=None)
MESSIDOR_Brest_Val = MESSIDOR(data_dir=MESSIDOR_data_dir_options['messidor_Brest-without_dilation'], mode='val', transform_=None)

In [5]:
MESSIDOR_train = ConcatDataset([MESSIDOR_pairs_train, MESSIDOR_Etienne_train, MESSIDOR_Brest_train])
MESSIDOR_Val = ConcatDataset([MESSIDOR_pairs_Val, MESSIDOR_Etienne_Val, MESSIDOR_Brest_Val])

In [6]:
MESSIDOR_2_train_loader = torch.utils.data.DataLoader(MESSIDOR_2_train, batch_size=1, shuffle=False, num_workers=4)
MESSIDOR_train_loader = torch.utils.data.DataLoader(MESSIDOR_train, batch_size=1, shuffle=False, num_workers=4)

MESSIDOR_2_Val_loader = torch.utils.data.DataLoader(MESSIDOR_2_Val, batch_size=1, shuffle=False, num_workers=4)
MESSIDOR_Val_loader = torch.utils.data.DataLoader(MESSIDOR_Val, batch_size=1, shuffle=False, num_workers=4)

In [7]:
save_path_meta = '/home/chong.tian/hc701/Pre_train_format/messidor2/train'
for i,(X,y) in enumerate(MESSIDOR_2_train_loader):
    X = X.squeeze().detach().cpu()
    X = transforms.ToPILImage()(X)
    label = int(y.tolist()[0])+1
    save_path = os.path.join(save_path_meta, 'class'+str(label))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    X.save(os.path.join(save_path, f"messidor_2_train_image_{i}.png"), format="PNG")

save_path_meta = '/home/chong.tian/hc701/Pre_train_format/messidor2/val'
for i,(X,y) in enumerate(MESSIDOR_2_Val_loader):
    X = X.squeeze().detach().cpu()
    X = transforms.ToPILImage()(X)
    label = int(y.tolist()[0])+1
    save_path = os.path.join(save_path_meta, 'class'+str(label))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    X.save(os.path.join(save_path, f"messidor_2_val_image_{i}.png"), format="PNG")

In [ ]:
save_path_meta = '/home/chong.tian/hc701/Pre_train_format/messidor/train'
for i,(X,y) in enumerate(MESSIDOR_train_loader):
    X = X.squeeze().detach().cpu()
    X = transforms.ToPILImage()(X)
    label = int(y.tolist()[0])+1
    save_path = os.path.join(save_path_meta, 'class'+str(label))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    X.save(os.path.join(save_path, f"messidor_train_image_{i}.png"), format="PNG")

save_path_meta = '/home/chong.tian/hc701/Pre_train_format/messidor/val'
for i,(X,y) in enumerate(MESSIDOR_Val_loader):
    X = X.squeeze().detach().cpu()
    X = transforms.ToPILImage()(X)
    label = int(y.tolist()[0])+1
    save_path = os.path.join(save_path_meta, 'class'+str(label))
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    X.save(os.path.join(save_path, f"messidor_val_image_{i}.png"), format="PNG")